In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')

# Load GeoJSON file
map_catalonia='county_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)

In [ ]:
# Rename column of interest and sort data by the latter
geo_data.rename(columns={'nomcomar': 'County_name'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='County_name')

In [ ]:
# Sum up votes for each county
votes_by_county = election_data.iloc[:, 21:].groupby(election_data['County_name']).sum()

In [ ]:
# Extract electoral census and add it to votes_by_county
electoral_census_by_county = list(election_data.iloc[:, 12].groupby(election_data['County_name']).sum())
votes_by_county['Electoral census'] = electoral_census_by_county 

In [ ]:
# Find the winner party for each county
winner_by_county=votes_by_county.iloc[:,:-2].idxmax(axis=1)
list_winners=list(winner_by_county)
# Add column with winner party
votes_by_county['winner']=list_winners
# Sort by county
votes_by_county.sort_values(by='County_name')


In [ ]:
# Merge county map and votes by county
merged_data = geo_data_sorted.merge(votes_by_county, on='County_name')
#merged_data.to_csv('election_results_county.csv', index=False)
merged_data

In [ ]:
# Plot the election results on a map of Catalonian counties

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='winner',  # Change to the column you want to visualize
    hover_name='County_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    color_discrete_sequence=["goldenrod", "darkred", "lightgreen"],
    title='Election results by county'
)

fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)

fig.show()